In [1]:
'''
import pandas as pd
import numpy as np

# 날짜 생성
date_range = pd.date_range(start='2023-01-01', end='2023-12-31')

# 초기값 설정
np.random.seed(45)  # 재현성을 위해 시드 설정
initial_special = 3092
initial_good = 2987
initial_bad = 2898

data = {
    'date': date_range,
    'special': [initial_special],
    'good': [initial_good],
    'bad': [initial_bad]
}

# 더미 데이터 생성
for i in range(1, len(date_range)):
    prev_special = data['special'][i-1]
    prev_good = data['good'][i-1]
    prev_bad = data['bad'][i-1]
    
    # special 품질 사과 수량 생성
    change_special = np.random.normal(loc=0, scale=100)
    new_special = prev_special + change_special
    if new_special < 2000:
        new_special = 2000 + abs(change_special)
    elif new_special > 4000:
        new_special = 4000 - abs(change_special)
    
    # good 품질 사과 수량 생성
    change_good = np.random.normal(loc=0, scale=100)
    new_good = prev_good + change_good
    if new_good < 2000:
        new_good = 2000 + abs(change_good)
    elif new_good > 4000:
        new_good = 4000 - abs(change_good)
    
    # bad 품질 사과 수량 생성
    change_bad = np.random.normal(loc=0, scale=100)
    new_bad = prev_bad + change_bad
    if new_bad < 2000:
        new_bad = 2000 + abs(change_bad)
    elif new_bad > 4000:
        new_bad = 4000 - abs(change_bad)
    
    data['special'].append(int(new_special))
    data['good'].append(int(new_good))
    data['bad'].append(int(new_bad))

# 데이터프레임 생성
df = pd.DataFrame(data)

# 데이터프레임 저장
df.to_csv("apple_stock.csv", index=False)

# 데이터 확인
print(df.head())
print(df.tail())
'''

'\nimport pandas as pd\nimport numpy as np\n\n# 날짜 생성\ndate_range = pd.date_range(start=\'2023-01-01\', end=\'2023-12-31\')\n\n# 초기값 설정\nnp.random.seed(45)  # 재현성을 위해 시드 설정\ninitial_special = 3092\ninitial_good = 2987\ninitial_bad = 2898\n\ndata = {\n    \'date\': date_range,\n    \'special\': [initial_special],\n    \'good\': [initial_good],\n    \'bad\': [initial_bad]\n}\n\n# 더미 데이터 생성\nfor i in range(1, len(date_range)):\n    prev_special = data[\'special\'][i-1]\n    prev_good = data[\'good\'][i-1]\n    prev_bad = data[\'bad\'][i-1]\n    \n    # special 품질 사과 수량 생성\n    change_special = np.random.normal(loc=0, scale=100)\n    new_special = prev_special + change_special\n    if new_special < 2000:\n        new_special = 2000 + abs(change_special)\n    elif new_special > 4000:\n        new_special = 4000 - abs(change_special)\n    \n    # good 품질 사과 수량 생성\n    change_good = np.random.normal(loc=0, scale=100)\n    new_good = prev_good + change_good\n    if new_good < 2000:\n        ne

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pmdarima import auto_arima
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# 데이터 로드
file_path = 'apple_stock.csv'
data = pd.read_csv(file_path)

# 날짜 형식을 datetime으로 변환
data['date'] = pd.to_datetime(data['date'])

# 최적의 하이퍼파라미터 찾기 함수 정의
def find_best_arima_params(data):
    model = auto_arima(data, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
    return model.order

# 각 품질별로 최적의 하이퍼파라미터 찾기
best_params = {}
for quality in ['special', 'good', 'bad']:
    best_order = find_best_arima_params(data[quality])
    best_params[quality] = best_order
    print(f'Best order for {quality}: {best_order}')


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=4317.655, Time=0.13 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4317.437, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=4319.377, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4319.356, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4315.488, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=4319.688, Time=0.15 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.383 seconds
Best order for special: (0, 1, 0)
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=4360.439, Time=0.23 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4357.287, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=4356.061, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4356.047, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4355.298, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=4357.913, Time=0.12 sec

Best model

In [3]:
best_params

{'special': (0, 1, 0), 'good': (0, 1, 0), 'bad': (0, 1, 0)}

In [4]:
# 예측 생성 함수 정의 (다음날 예측)
def generate_forecast(data, best_params):
    last_date = data['date'].max()
    next_date = last_date + pd.Timedelta(days=1)
    forecast_data = {'date': [next_date]}

    for quality in ['special', 'good', 'bad']:
        best_order = best_params[quality]
        model = sm.tsa.ARIMA(data[quality], order=best_order)
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=1)
        forecast = np.round(forecast).astype(int)  # 정수형으로 변환
        forecast_data[quality] = forecast

    forecast_df = pd.DataFrame(forecast_data)
    return forecast_df


In [5]:
# JupyterDash 앱 초기화
app = JupyterDash(__name__)

# 레이아웃 정의
app.layout = html.Div([
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=data['date'].min(),
        end_date=data['date'].max(),
        display_format='YYYY-MM-DD',
        style={'backgroundColor': '#E0F7FA', 'color': '#000000'}
    ),
    dcc.Graph(id='line-chart', style={'backgroundColor': '#E0F7FA'}),
    dcc.Graph(id='pie-chart', style={'backgroundColor': '#E0F7FA'}),
    dcc.Graph(id='forecast-chart', style={'backgroundColor': '#E0F7FA'})
], style={'backgroundColor': '#E0F7FA'})


c:\Users\dhshs\anaconda3\Lib\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [6]:
# 콜백 함수 정의
@app.callback(
    [Output('line-chart', 'figure'),
     Output('pie-chart', 'figure'),
     Output('forecast-chart', 'figure')],
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_charts(start_date, end_date):
    if not start_date or not end_date:
        return {}, {}, {}

    # 선택한 날짜 범위에 따른 데이터 필터링
    filtered_data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

    # 라인 차트 생성
    line_fig = px.line(filtered_data, x='date', y=['special', 'good', 'bad'], 
                       labels={'value': '수량', 'variable': '품질'},
                       title='날짜별 품질별 사과 수량')
    line_fig.update_layout(xaxis_title='날짜', yaxis_title='수량', template='plotly', plot_bgcolor='#E0F7FA')

    # 원형 차트 생성
    pie_data = filtered_data[['special', 'good', 'bad']].sum().reset_index()
    pie_data.columns = ['quality', 'quantity']
    pie_fig = px.pie(pie_data, values='quantity', names='quality', title='품질별 사과 수량 비율')
    pie_fig.update_layout(template='plotly', plot_bgcolor='#E0F7FA')

    # 예측 데이터 생성
    forecast_data = generate_forecast(data, best_params)
    
    # 예측 데이터 막대그래프 생성
    forecast_fig = px.bar(forecast_data.melt(id_vars='date', value_vars=['special', 'good', 'bad']),
                          x='variable', y='value', color='variable', 
                          labels={'value': '수량', 'variable': '품질'},
                          title='다음날 품질별 사과 수량 예측')
    forecast_fig.update_layout(xaxis_title='품질', yaxis_title='수량', template='plotly', plot_bgcolor='#E0F7FA')

    return line_fig, pie_fig, forecast_fig

# 대시보드 실행
app.run_server(mode='inline')


c:\Users\dhshs\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\dhshs\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

